In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import os
import glob


from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE, ProjectionCARE
from helpers import save_8bit_tiff_imagej_compatible

try:
    from pathlib import Path
    Path().expanduser()
except (ImportError, AttributeError):
        from pathlib2 import Path

try:
        import tempfile
        tempfile.TemporaryDirectory
except (ImportError, AttributeError):
       from backports import tempfile    
        
import cv2 

from skimage import exposure

import time

os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


**Movie 3 + new model importation**

In [2]:
basedir='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a' 

basedirResults2D='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a/Projected'
basedirResults3D='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a/Restored'
basedirResults2Dextended='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a/Projected/Projected'
basedirResults3Dextended='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a/Restored/Restored'

Model_Dir='/home/sancere/NextonDisk_1/Lucas_Model_to_use/CARE/'

In [3]:
RestorationModel ='CARE_restoration_SpinWideFRAP4_Bin1'
ProjectionModel ='CARE_projection_SpinWideFRAP4_Bin1'

RestorationModel = CARE(config = None, name = RestorationModel, basedir = Model_Dir)
ProjectionModel = ProjectionCARE(config = None, name = ProjectionModel, basedir = Model_Dir) 





Loading network weights from 'weights_best.h5'.








Loading network weights from 'weights_best.h5'.


In [4]:
Path(basedirResults3D).mkdir(exist_ok = True)
Path(basedirResults2D).mkdir(exist_ok = True)

Raw_path = os.path.join(basedir, '*TIF') #tif or TIF be careful

axes = 'ZYX'  #projection axes : 'YX'

filesRaw = glob.glob(Raw_path)

In [5]:
for fname in filesRaw:
       if  os.path.exists(fname) == True :
             if   os.path.exists(basedirResults2Dextended + '_' + os.path.basename(fname)) == False  or os.path.exists(basedirResults3Dextended + os.path.basename(fname)) == False : 
                print(fname)
                y = imread(fname)
                restored = RestorationModel.predict(y, axes, n_tiles = (1,2,4)) #n_tiles is for the decomposition of the image in (z,y,x). (1,2,2) will work with light images. Less tiles we have, faster the calculation is 
                projection = ProjectionModel.predict(restored, axes, n_tiles = (1,1,1)) #n_tiles is for the decomposition of the image in (z,y,x). There is overlapping in the decomposition wich is managed by the program itself
                axes_restored = axes.replace(ProjectionModel.proj_params.axis, '')
                restored = restored.astype('uint8') # if prediction and projection running at the same time
                #restored = restored.astype('uint16') # if projection training set creation or waiting for a future projection 
                projection = projection.astype('uint8')
                save_tiff_imagej_compatible((basedirResults3Dextended  + os.path.basename(fname)) , restored, axes)
                save_tiff_imagej_compatible((basedirResults2Dextended + '_' + os.path.basename(fname)) , projection, axes_restored)

/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a/200626_Nts_cadG_mov1_w2CSU-TRIPLE-491_s1_t211.TIF


100%|██████████| 8/8 [00:13<00:00,  1.73s/it]   


/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a/200626_Nts_cadG_mov1_w2CSU-TRIPLE-491_s7_t93.TIF


100%|██████████| 8/8 [00:04<00:00,  1.95it/s]   


/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a/200626_Nts_cadG_mov1_w2CSU-TRIPLE-491_s3_t74.TIF


...

100%|██████████| 8/8 [00:04<00:00,  1.93it/s]   


/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a/200626_Nts_cadG_mov1_w2CSU-TRIPLE-491_s8_t112.TIF


100%|██████████| 8/8 [00:04<00:00,  1.94it/s]   


/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/Nts4a/200626_Nts_cadG_mov1_w2CSU-TRIPLE-491_s4_t74.TIF


100%|██████████| 8/8 [00:04<00:00,  1.94it/s]   


**Movie 4**

In [7]:
basedir='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c' 

basedirResults2D='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c/Projected'
basedirResults3D='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c/Restored'
basedirResults2Dextended='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c/Projected/Projected'
basedirResults3Dextended='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c/Restored/Restored'

Model_Dir='/home/sancere/NextonDisk_1/Lucas_Model_to_use/CARE/'

In [8]:
Path(basedirResults3D).mkdir(exist_ok = True)
Path(basedirResults2D).mkdir(exist_ok = True)

Raw_path = os.path.join(basedir, '*TIF') #tif or TIF be careful

axes = 'ZYX'  #projection axes : 'YX'

filesRaw = glob.glob(Raw_path)

In [9]:
for fname in filesRaw:
       if  os.path.exists(fname) == True :
             if   os.path.exists(basedirResults2Dextended + '_' + os.path.basename(fname)) == False  or os.path.exists(basedirResults3Dextended + os.path.basename(fname)) == False : 
                print(fname)
                y = imread(fname)
                restored = RestorationModel.predict(y, axes, n_tiles = (1,2,4)) #n_tiles is for the decomposition of the image in (z,y,x). (1,2,2) will work with light images. Less tiles we have, faster the calculation is 
                projection = ProjectionModel.predict(restored, axes, n_tiles = (1,1,1)) #n_tiles is for the decomposition of the image in (z,y,x). There is overlapping in the decomposition wich is managed by the program itself
                axes_restored = axes.replace(ProjectionModel.proj_params.axis, '')
                restored = restored.astype('uint8') # if prediction and projection running at the same time
                #restored = restored.astype('uint16') # if projection training set creation or waiting for a future projection 
                projection = projection.astype('uint8')
                save_tiff_imagej_compatible((basedirResults3Dextended + '_' + os.path.basename(fname)) , restored, axes)
                save_tiff_imagej_compatible((basedirResults2Dextended + '_' + os.path.basename(fname)) , projection, axes_restored)

/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c/200625_pzKO_ubi-cad_mov2_w2CSU-TRIPLE-491_s16_t50.TIF


100%|██████████| 8/8 [00:04<00:00,  1.94it/s]   


/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c/200625_pzKO_ubi-cad_mov2_w2CSU-TRIPLE-491_s14_t128.TIF


100%|██████████| 8/8 [00:04<00:00,  1.93it/s]   


/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c/200625_pzKO_ubi-cad_mov2_w2CSU-TRIPLE-491_s14_t37.TIF


...

100%|██████████| 8/8 [00:04<00:00,  1.92it/s]   


/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c/200625_pzKO_ubi-cad_mov2_w2CSU-TRIPLE-491_s21_t35.TIF


100%|██████████| 8/8 [00:04<00:00,  1.94it/s]   


/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Maria_movies/pzKO2c/200625_pzKO_ubi-cad_mov2_w2CSU-TRIPLE-491_s16_t181.TIF


100%|██████████| 8/8 [00:04<00:00,  1.92it/s]   


**Movie 5**

In [10]:
basedir='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Restored' 

basedirResults2D='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Projected2'
#basedirResults3D='/Restored2'
basedirResults2Dextended='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Projected2/Projected2'
#basedirResults3Dextended='/Restored2/Restored2'

Model_Dir='/home/sancere/NextonDisk_1/Lucas_Model_to_use/CARE/'

In [11]:
#RestorationModel ='CARE_restoration_Borealis_Bin2_AudeData'
ProjectionModel ='CARE_projection_Borealis_Bin2_AudeData_2_4'

#RestorationModel = CARE(config = None, name = RestorationModel, basedir = Model_Dir)
ProjectionModel = ProjectionCARE(config = None, name = ProjectionModel, basedir = Model_Dir) 


Loading network weights from 'weights_best.h5'.


In [12]:
Path(basedirResults3D).mkdir(exist_ok = True)
Path(basedirResults2D).mkdir(exist_ok = True)

Raw_path = os.path.join(basedir, '*TIF') #tif or TIF be careful

axes = 'ZYX'  #projection axes : 'YX'

filesRaw = glob.glob(Raw_path)

In [13]:
for fname in filesRaw:
       if  os.path.exists(fname) == True :
             if   os.path.exists(basedirResults2Dextended + '_' + os.path.basename(fname)) == False  or os.path.exists(basedirResults3Dextended + os.path.basename(fname)) == False : 
                print(fname)
                y = imread(fname)
                restored = y
                #restored = RestorationModel.predict(y, axes, n_tiles = (1,2,4)) #n_tiles is for the decomposition of the image in (z,y,x). (1,2,2) will work with light images. Less tiles we have, faster the calculation is 
                projection = ProjectionModel.predict(restored, axes, n_tiles = (1,1,1)) #n_tiles is for the decomposition of the image in (z,y,x). There is overlapping in the decomposition wich is managed by the program itself
                axes_restored = axes.replace(ProjectionModel.proj_params.axis, '')
                #restored = restored.astype('uint8') # if prediction and projection running at the same time
                #restored = restored.astype('uint16') # if projection training set creation or waiting for a future projection 
                projection = projection.astype('uint8')
                #save_tiff_imagej_compatible((basedirResults3Dextended + '_' + os.path.basename(fname)) , restored, axes)
                save_tiff_imagej_compatible((basedirResults2Dextended + '_' + os.path.basename(fname)) , projection, axes_restored)

/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Restored/RestoredEcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s9_t50.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Restored/RestoredEcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s2_t11.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Restored/RestoredEcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s6_t60.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Restored/RestoredEcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s7_t53.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Restored/RestoredEcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s16_t12.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cal

**Movie 6**

In [5]:
basedir='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626' 

basedirResults2D='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Projected3'
#basedirResults3D='/Restored2'
basedirResults2Dextended='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/Projected3/Projected3'
#basedirResults3Dextended='/Restored2/Restored2'

Model_Dir='/home/sancere/NextonDisk_1/Lucas_Model_to_use/CARE/'

In [6]:
#RestorationModel ='CARE_restoration_Borealis_Bin2_AudeData'
ProjectionModel ='CARE_projection_Borealis_Bin2_AudeData_2_2'

#RestorationModel = CARE(config = None, name = RestorationModel, basedir = Model_Dir)
ProjectionModel = ProjectionCARE(config = None, name = ProjectionModel, basedir = Model_Dir) 


Loading network weights from 'weights_best.h5'.


In [7]:
#Path(basedirResults3D).mkdir(exist_ok = True)
Path(basedirResults2D).mkdir(exist_ok = True)

Raw_path = os.path.join(basedir, '*TIF') #tif or TIF be careful

axes = 'ZYX'  #projection axes : 'YX'

filesRaw = glob.glob(Raw_path)

In [9]:
for fname in filesRaw:
       if  os.path.exists(fname) == True :
             if   os.path.exists(basedirResults2Dextended + '_' + os.path.basename(fname)) == False : 
                print(fname)
                y = imread(fname)
                restored = y
                #restored = RestorationModel.predict(y, axes, n_tiles = (1,2,4)) #n_tiles is for the decomposition of the image in (z,y,x). (1,2,2) will work with light images. Less tiles we have, faster the calculation is 
                projection = ProjectionModel.predict(restored, axes, n_tiles = (1,1,1)) #n_tiles is for the decomposition of the image in (z,y,x). There is overlapping in the decomposition wich is managed by the program itself
                axes_restored = axes.replace(ProjectionModel.proj_params.axis, '')
                #restored = restored.astype('uint8') # if prediction and projection running at the same time
                #restored = restored.astype('uint16') # if projection training set creation or waiting for a future projection 
                projection = projection.astype('uint8')
                #save_tiff_imagej_compatible((basedirResults3Dextended + '_' + os.path.basename(fname)) , restored, axes)
                save_tiff_imagej_compatible((basedirResults2Dextended + '_' + os.path.basename(fname)) , projection, axes_restored)

/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/EcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s4_t44.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/EcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s9_t48.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/EcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s12_t13.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/EcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s3_t43.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/EcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s7_t11.TIF
/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/a_maugarny-cales/20200626/EcadG_phmG4TS_spokRNAi_48h29C1_w2CSU-488_s7_t29.TIF
/run/user/1000/gvfs/s